In [1]:
import torch
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import CIFAR100
from typing import Optional, Callable
import os
import timm
import numpy as np
import pandas as pd
from torchvision.transforms import v2
from torch.backends import cudnn
from torch import GradScaler
from torch import optim
from tqdm import tqdm

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [2]:
list_name = [
    'caformer_s18',
    'caformer_m36',
    'caformer_b36',
    'maxvit_small_tf_224',
    'maxvit_base_tf_224'
]

In [3]:
device = torch.device('cuda')
cudnn.benchmark = True
pin_memory = True
enable_half = True  # Disable for CPU, it is slower!
scaler = GradScaler(device, enabled=enable_half)

In [4]:
class SimpleCachedDataset(Dataset):
    def __init__(self, dataset, transform):
        self.data = dataset.data
        self.targets = dataset.targets
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        return self.transform(self.data[i]), self.targets[i]

In [5]:
class CIFAR100_noisy_fine(Dataset):
    """
    See https://github.com/UCSC-REAL/cifar-10-100n, https://www.noisylabels.com/ and `Learning with Noisy Labels
    Revisited: A Study Using Real-World Human Annotations`.
    """

    def __init__(
        self, root: str, train: bool, transform: Optional[Callable], download: bool
    ):
        cifar100 = CIFAR100(
            root=root, train=train, transform=None, download=download
        )
        data, targets = tuple(zip(*cifar100))

        if train:
            noisy_label_file = os.path.join(root, "/kaggle/input/fii-atnn-2025-project-noisy-cifar-100/fii-atnn-2024-project-noisy-cifar-100/CIFAR-100-noisy.npz")
            if not os.path.isfile(noisy_label_file):
                raise FileNotFoundError(
                    f"{type(self).__name__} need {noisy_label_file} to be used!"
                )

            noise_file = np.load(noisy_label_file)
            if not np.array_equal(noise_file["clean_label"], targets):
                raise RuntimeError("Clean labels do not match!")
            targets = noise_file["noisy_label"]

        self.data = data
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, i: int):
        return self.transform(self.data[i]), self.targets[i]

In [6]:
train_transform = v2.Compose([
    v2.Resize((224, 224)),
    v2.RandAugment(num_ops=6, magnitude=9),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=(0.5071, 0.4865, 0.4409), std=(0.2673, 0.2564, 0.2762), inplace=True),
])
test_transform = v2.Compose([
    v2.Resize((224, 224)),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=(0.5071, 0.4865, 0.4409), std=(0.2673, 0.2564, 0.2762), inplace=True),
])
test_set = CIFAR100_noisy_fine('/kaggle/input/fii-atnn-2025-project-noisy-cifar-100/fii-atnn-2024-project-noisy-cifar-100', download=False, train=False, transform=test_transform)
test_set = SimpleCachedDataset(test_set, transform=test_transform)
test_loader = DataLoader(test_set, batch_size=8, pin_memory=pin_memory)

In [7]:
def create_base_model(name):
    return timm.create_model(
        name,
        pretrained=True,
        num_classes=100
    )

In [8]:
@torch.inference_mode()
def inference_to_get_test_labels():
    models = []
    for name in list_name:
        model = create_base_model(name)
        model_path = f'/kaggle/input/ansamblu-v3/ans/best_{name}.pth'
        state = torch.load(model_path, map_location=device)
        model.load_state_dict(state)
        model.to(device)
        model.eval()
        models.append(model)

    all_preds = []
    for inputs, _ in tqdm(test_loader):
        inputs = inputs.to(device, non_blocking=True)

        with torch.autocast(device_type=device.type, enabled=enable_half):
            logits = [m(inputs) for m in models]
            summed = torch.stack(logits, dim=0).sum(dim=0)

        preds = summed.argmax(dim=1)          # [B]
        all_preds.extend(preds.cpu().tolist())

    return all_preds

In [9]:
test_loader = DataLoader(test_set, batch_size=8, pin_memory=pin_memory)
test_pred = inference_to_get_test_labels()

model.safetensors:   0%|          | 0.00/105M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/225M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/395M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/276M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/479M [00:00<?, ?B/s]

100%|██████████| 1250/1250 [06:36<00:00,  3.15it/s]


In [10]:
print(len(test_pred))

10000


In [11]:
test_loader = DataLoader(test_set, batch_size=1, pin_memory=pin_memory)

acc = 0
cont = 0
for _, targ in test_loader:
    if targ == test_pred[cont]:
        acc += 1
    cont += 1
print(acc/len(test_pred))

0.8429


In [12]:
data = {
    "ID": [],
    "target": []
}

for i, label in enumerate(test_pred):
    data["ID"].append(i)
    data["target"].append(label)

df = pd.DataFrame(data)
df.to_csv(f"./submission.csv", index=False)